# Introduction to T<sub>c</sub>1D (Inverse mode)

This is a Jupyter notebook, an interactive web application that can be used to run numerical simulations. In this notebook we will explore using the T<sub>c</sub>1D software in inverse mode.

# Inversion of Thermochronological Data with T<sub>c</sub>1D

Thermochronology provides quantitative constraints on the cooling history of rocks through geological time. These data are used to infer the timing and rate of processes such as exhumation and burial. However, the relationship between the geological parameters (e.g., erosion history) and the observed thermochronological ages is complex and non-linear. Therefore, you need to solve an inverse problem: What are the ranges of model parameters (e.g., erosion rate, timing of erosion, etc.) that best reproduce the observed data?

## What is an Inversion?

An inversion explores possible model parameters and selects those that, when used in forward simulations, generate synthetic data consistent with the observations. In practice, we compare observed and predicted thermochronological ages to quantify the model misfit. Inverse problems are often non-unique and can present multiple acceptable solutions, which is why we must explore the parameter space systematically.

## Inversion Methods in T<sub>c</sub>1D

This notebook illustrates two popular approaches for exploring the parameter space and performing inversions with the T<sub>c</sub>1D software:

### 1. Neighbourhood Algorithm (NA)

- A stochastic, adaptive sampling method.
- It iteratively explores the parameter space by favoring regions of lower misfit.
- It efficiently identifies acceptable models and only approximates the posterior probability distribution.

### 2. Markov Chain Monte Carlo (MCMC)

- A Bayesian, probabilistic method based on random walks.
- It samples the parameter space proportionally to the likelihood of each model.
- It allows full statistical characterization of the uncertainties but is often computationally more expensive.

## Objective of this Notebook

We will use T<sub>c</sub>1D to perform both NA and MCMC inversions on a simple dataset. The goal is to illustrate how these methods work and how to visualize and interpret the results.

**⚠️ Disclaimer ⚠️**

The inversions performed in this notebook are simplified and use a limited number of parameters, reduced model complexity, and small sampling sizes. This is done intentionally to ensure rapid execution and allow interactive exploration within a Jupyter notebook environment. The results shown here are purely illustrative and do not represent a scientifically valid interpretation of real thermochronological datasets. For rigorous scientific applications, inversions should be performed with appropriately complex models, realistic parameter ranges, and sufficient sampling to ensure robust exploration of the parameter space.

## Attribution

If you use plots produced by this software, please cite the following:

- D. Whipp. (2022). HUGG/TC1D: v0.1 (v0.1). Zenodo. https://doi.org/10.5281/zenodo.7124272.

The age prediction software used for calculating apatite and zircon (U-Th)/He and apatite fission-track ages was written by Richard Ketcham at the University of Texas, USA. Results published using this software should cite the articles below:

- Ketcham, R. A., Donelick, R. A., & Carlson, W. D.: Variability of apatite fission-track annealing kinetics III: Extrapolation to geological time scales. American Mineralogist, 84, 1235-1255, doi: [10.2138/am-1999-0903](https://doi.org/10.2138/am-1999-0903), 1999.

- Ketcham, R. A., Mora, A., and Parra, M.: Deciphering exhumation and burial history with multi-sample down-well thermochronometric inverse modelling, Basin Res., 30, 48-64, [10.1111/bre.12207](https://doi.org/10.1111/bre.12207), 2018.

The inversion methods implemented in T<sub>c</sub>1D are based on the following references:

- Sambridge, M.: Geophysical inversion with a neighbourhood algorithm – I. Searching a parameter space, *Geophys. J. Int.*, 138, 479–494, [10.1046/j.1365-246x.1999.00876.x](https://doi.org/10.1046/j.1365-246x.1999.00876.x), 1999.

- Sambridge, M.: Geophysical inversion with a neighbourhood algorithm – II. Appraising the ensemble, *Geophys. J. Int.*, 138, 727–746, [10.1046/j.1365-246x.1999.00900.x](https://doi.org/10.1046/j.1365-246x.1999.00900.x), 1999.

- Gilks, W.R., Richardson, S., & Spiegelhalter, D.J. (eds.): *Markov Chain Monte Carlo in Practice*. Chapman & Hall/CRC, London, 1996.

- Foreman-Mackey, D., Hogg, D.W., Lang, D., & Goodman, J.: emcee: The MCMC Hammer, *Publ. Astron. Soc. Pac.*, 125, 306–312, [10.1086/670067](https://doi.org/10.1086/670067), 2013.

# Using this notebook

It is easy to get started using this notebook. Below you will find some general information about the notebook environment and some examples of how to perform inversions using NA and MCMC.

## Using a Jupyter notebook

A Jupyter notebook is a document that combines rich text formatting (like that in a word processor or website) with programming language code. The notebook itself is divided into blocks called cells that have a defined cell type, which means a cell can either contain rich text, code, or raw unformatted text (but not a mix). For us, the main concern will be code cells and how to run them, as that will be the way to run inverse models and/or produce a plot.

### Running a code cell

There are two options for running a code cell.

1. Click on the cell containing code and press one of the following key combinations:

    - <kbd>shift</kbd> + <kbd>enter</kbd> or
    - <kbd>shift</kbd> + <kbd>return</kbd>

    On a Mac keyboard the <kbd>shift</kbd> keys have arrows pointing up and the <kbd>return</kbd> is on the far right with a bent arrow pointing left.

2. Select a cell containing code and press the play button (▶︎) in the toolbar.

Let's test this out with an example below, just to make sure the environment is working. Click on the code cell below and then press <kbd>shift</kbd> + <kbd>enter</kbd> or <kbd>shift</kbd> + <kbd>return</kbd> to run it.

In [ ]:
print(f"According to my Jupyter notebook, 1 + 1 = {1 + 1}")

## Using Binder

[Binder](https://mybinder.org/) is a cloud computing platform that provides the computing used to run a Jupyter notebook free of charge. You are most likely using Binder right now if you have opened this notebook and the code example above works. You don't really need to know much about Binder in order to use it, however, there is one important note about Binder: **Your session will die and your notebook will stop functioning after about 10 minutes of inactivity**. This means you may lose any progress you have made in the notebook after this time elapses. If you want to keep your session alive, be sure to run at least one code cell every 10 minutes.

You can find more information about Binder in the [Binder user guide](https://mybinder.readthedocs.io/en/latest/index.html).

# Obligatory starting point

In order to run either of the inversion examples below, **you must run the code cell below first**! This will load the libraries needed for running the inverse models. Note that lines starting with the `#` character are comment lines that can be used for documentation, but are not executed as Python commands.

In [ ]:
# import libraries we need
from IPython.display import Image, display
import pandas as pd
import subprocess

## Our Synthetic Thermochronology Dataset

The table obtained with the command below presents a simple synthetic dataset that will be used for demonstration purposes in this notebook. It includes a combination of commonly used low-temperature thermochronometers:

- **AHe**: Apatite (U-Th)/He
- **AFT**: Apatite Fission Tracks
- **ZHe**: Zircon (U-Th)/He
- **ZFT**: Zircon Fission Tracks

For each sample, we provide:

- The measured age (in millions of years, Ma)
- The standard deviation of the age (representing uncertainty)
- The effective Uranium (eU) concentration for (U-Th)/He samples
- The grain radius for (U-Th)/He samples
- A sample ID for identification

**ℹ️ Note**

Missing values (`NaN`) correspond to information not relevant for the specific thermochronometer (e.g., eU and grain radius are not applicable to fission track samples).

You can run the code cell below to see the table data.

In [ ]:
# load from file data
data_file = "tc1d/csv/sample_data.csv"
df = pd.read_csv(data_file, index_col="Sample ID")
df

# Running a T<sub>c</sub>1D inversion using the Neighbourhood algorithm (NA)

With the necessary background out of the way we can now move forward to running a first inversion.

The following code cell performs an inversion of the synthetic dataset using the Neighbourhood Algorithm (NA) implemented in T<sub>c</sub>1D.

**⏱️ Runtime** : The process should take few minutes. Binder typically culls idle sessions after 10 minutes, but a running cell usually keeps the session alive. Keep this tab open and prevent your device from sleeping.

The inversion is launched via a command-line call to the `tc1d_cli.py` script in the code cell below, specifying:

- The erosion model type (`--ero-type`)
- The erosion history parameters (`--ero-option1` to `--ero-option4`)
- The total model time and time step (`--time`, `--dt`)
- The input dataset (`--obs-age-file`)
- The inversion mode (`NA` in this case)

Once the inversion is complete, a series of plots are generated and saved in the current directory. The cell that follows then displays these plots directly in the notebook for easy inspection. These plots include:

- **Misfit plot**: model misfit evolution
- **Scatter plot**: parameter space exploration
- **Covariance matrix**: parameter correlation structure
- **Voronoi plot**: visualization of NA sampling

### Example Inversion Parameters

The inversion performed in this example uses the following erosion history parameters and model configuration:

| Parameter         | Value(s)      | Description                                  |
|-------------------|---------------|----------------------------------------------|
| `--ero-type`      | 2             | Erosion model type                           |
| `--ero-option1`   | 0.0 to 15.0   | First erosion magnitude: parameter range (km)    |
| `--ero-option2`   | 6.0           | First erosion event: fixed time (Ma) between 0 and 6 Myrs        |
| `--ero-option3`   | 0.0 to 15.0   | Second erosion magnitude: parameter range (km)    |
| `--ero-option4`   | 12.0          | Second erosion event: fixed time (Ma) between 6 and 12 Myrs       |
| `--time`          | 12.0          | Total model time (Ma)                        |
| `--dt`            | 20000         | Time step for forward model (years)          |
| `--obs-age-file`  | `tc1d/sample_data.csv` | Path to input dataset                     |
| `--inverse-mode`  | NA            | Inversion mode: Neighbourhood Algorithm      |

**ℹ️ Note**

Some parameters are defined as ranges (e.g., `0.0 to 15.0`). This informs the inversion algorithm to explore possible values within those limits. Other parameters are fixed.

### About Erosion Model Type 2

Erosion model **Type 2** in T<sub>c</sub>1D corresponds to a constant-rate erosion model with stepwise changes at specified times.
This setup allows up to three distinct erosion phases, each at a constant rate, separated by sharp transitions.

The erosion history parameters in this example (`--ero-option1` to `--ero-option4`) correspond to:

- `--ero-option1`: exhumation magnitude *m₁* (km) for the first phase
- `--ero-option2`: transition time *t₁* (Myr) at which erosion rate changes
- `--ero-option3`: exhumation magnitude *m₂* (km) for the second phase
- `--ero-option4`: (optional) second transition time *t₂* (Myr) defining a third phase

Erosion rates are calculated as the magnitude of each phase divided by its duration. This formulation makes it possible to simulate stepwise pulses of exhumation or burial.

For additional details and mathematical formulations, see the [TC1D documentation](https://tc1d.readthedocs.io/en/latest/erosion-models.html#type-2-constant-rate-s-with-step-function-change-s-at-specified-time-s).

### Neighbourhood Algorithm (NA) Settings

In the T<sub>c</sub>1D code, the NA is configured using the `NASearcher` class from the [`neighpy` package](https://neighpy.readthedocs.io). The main parameters controlling the inversion are defined directly in the `tc1d.py` script.

Below is a description of the key NA settings used in this example:

| Parameter  | Value | Description                                                      |
|:----------:|:-----:|:-----------------------------------------------------------------|
| `ns`       | 30    | Number of samples per iteration (controls how many new models are generated in each iteration) |
| `nr`       | 15    | Number of cells to resample (determines how many of the lowest misfit regions are preferentially explored) |
| `ni`       | 60   | Number of initial random samples (controls the size of the initial exploration phase) |
| `n`        | 10     | Number of iterations of the algorithm (total number of NA steps performed) |

The parameter `bounds` defines the limits of the parameter space for the inversion (ranges for erosion events in this case).

**❗️ IMPORTANT**

To modify these NA settings, you need to edit the source code of the `tc1d.py` script directly. These parameters are hard-coded for simplicity in this notebook example.

In [ ]:
# run NA inversion
command = [
    "python", "tc1d/tc1d_cli.py",
    "--ero-type", "2",
    "--ero-option1", "0.0", "15.0",
    "--ero-option2", "6.0",
    "--ero-option3", "0.0", "15.0",
    "--ero-option4", "12.0",
    "--time", "12.0",
    "--dt", "20000",
    "--obs-age-file", "tc1d/sample_data.csv",
    "--inverse-mode", "NA"
]

subprocess.run(command)

In [ ]:
display(Image("NA_misfit.png"))
display(Image("NA_scatter.png"))
display(Image("NA_covariance_matrix.png"))
display(Image("NA_voronoi.png"))

### Interpretation of the NA Inversion Results

The following plots summarize the output of the Neighbourhood Algorithm (NA) inversion performed with T<sub>c</sub>1D:

1. **Misfit Plot (`NA_misfit.png`)**

   This figure shows the evolution of the model misfit throughout the inversion process.  
   - The first part (left of the dashed line) corresponds to the initial random search.  
   - The second part shows the focused Neighbourhood Search, where the algorithm concentrates on low-misfit regions.  
   A decreasing trend indicates that the inversion progressively identifies better-fitting models.

2. **Scatter Plot of the Sampled Parameter Space (`NA_scatter.png`)**

   This scatter plot shows the sampled parameter space for two erosion parameters (`ero_option1` and `ero_option3`).  
   - The color scale represents the model misfit (lower is better).  
   - The histograms along the top and right axes show the distribution of tested parameter values.  
   - The red cross indicates the best-fitting model identified during the inversion.  
   Clustering of low-misfit points reveals regions of acceptable solutions.

3. **Covariance Matrix of Inverted Parameters (`NA_covariance_matrix.png`)**

   This matrix quantifies the covariance between the inverted parameters.  
   - Large positive values along the diagonal correspond to the variance of each parameter.  
   - Off-diagonal values indicate the degree of correlation between parameters.  
   - For example, a negative covariance suggests an inverse relationship between two parameters.  
   Understanding parameter correlation is essential for interpreting inversion uncertainties.

4. **Voronoi Diagram of the Parameter Space (`NA_voronoi.png`)**

   This plot provides a geometrical visualization of the parameter space partitioning based on the sampled models.  
   - Each polygon represents a region of the parameter space associated with a specific sampled model.  
   - The green marker indicates the location of the best-fitting model.  
   This plot highlights how the NA efficiently focuses exploration on promising regions of the parameter space.

## Forward Modeling with Best-Fit Parameters

After completing the inversion, we can now run T<sub>c</sub>1D in forward mode using the best-fit parameters identified by the Neighbourhood Algorithm.

This allows us to:

- Simulate the thermal evolution of the model based on these optimal parameters.
- Predict synthetic thermochronological ages.
- Compare these predicted ages to the original synthetic dataset.

This comparison provides a direct assessment of how well the inverted model reproduces the observed data and allows us to evaluate the quality of the inversion.

In [ ]:
# run forward model
command = [
    "python", "tc1d/tc1d_cli.py",
    "--ero-type", "2",
    "--ero-option1", "2.58",# NA inversion should converge around this value
    "--ero-option2", "6.0",
    "--ero-option3", "6.02",# NA inversion should converge around this value
    "--ero-option4", "12.0",
    "--time", "12.0",
    "--dt", "20000",
    "--obs-age-file", "tc1d/sample_data.csv",
    "--save-plots",
]

subprocess.run(command)

display(Image("png/cooling_hist.png"))

# Running T<sub>c</sub>1D inversion using the MCMC algorithm (MCMC)

The following code cell performs an inversion of the synthetic dataset using the Markov Chain Monte Carlo (MCMC) algorithm implemented in T<sub>c</sub>1D.

**⏱️ Runtime**: The process should take 20-40 minutes. Binder typically culls idle sessions after 10 minutes, but a running cell usually keeps the session alive. Keep this tab open and prevent your device from sleeping.

The inversion is launched via a command-line call to the `tc1d_cli.py` script, specifying:

- The erosion model type (`--ero-type`)
- The erosion history parameters (`--ero-option1` to `--ero-option4`)
- The total model time and time step (`--time`, `--dt`)
- The input dataset (`--obs-age-file`)
- The inversion mode (`MCMC` in this case)

Once the inversion is complete, a series of plots are generated and saved in the current directory. The cell that follows then displays these plots directly in the notebook for easy inspection. These include:

- **Misfit plot**: model misfit evolution
- **Chains plot**: walker chains for each parameter (diagnostic of convergence)
- **Corner plot**: posterior distribution of parameters and correlations
- **Scatter plot**: parameter space exploration

### Example Inversion Parameters

The inversion performed in this example uses the following erosion history parameters and model configuration.

**ℹ️ Note**

These are the same as for the NA example, so the results would be expected to be similar, even though the two approaches differ.

Parameters we consider include:

| Parameter         | Value(s)      | Description                                           |
|-------------------|---------------|-------------------------------------------------------|
| `--ero-type`      | 2             | Erosion model type                                    |
| `--ero-option1`   | 0.0 to 15.0   | First erosion magnitude: parameter range (km)         |
| `--ero-option2`   | 6.0           | First erosion event: fixed time (Ma)                  |
| `--ero-option3`   | 0.0 to 15.0   | Second erosion magnitude: parameter range (km)        |
| `--ero-option4`   | 12.0          | Second erosion event: fixed time (Ma)                 |
| `--time`          | 12.0          | Total model time (Ma)                                 |
| `--dt`            | 20000         | Time step for forward model (years)                   |
| `--obs-age-file`  | `tc1d/sample_data.csv` | Path to input dataset                     |
| `--inverse-mode`  | MCMC          | Inversion mode: Markov Chain Monte Carlo              |

**ℹ️ Note**

Some parameters are defined as ranges (e.g., `0.0 to 15.0`). This informs the inversion algorithm to explore possible values within those limits. Other parameters are fixed.

### MCMC Settings

In the T<sub>c</sub>1D implementation, MCMC is handled using the [`emcee` package](https://emcee.readthedocs.io/) with optional parallelization using MPI.

The following settings control the MCMC run:

| Parameter   | Value  | Description                                                   |
|:-----------:|:------:|:--------------------------------------------------------------|
| `nwalkers`  | 8      | Number of walkers (independent chains)                        |
| `nsteps`    | 300    | Number of steps per walker                                    |
| `discard`   | 50     | Number of initial steps to discard as "burn-in"               |
| `thin`      | 3      | Thinning factor: only every 3rd step is retained in the chain |

**❗️ IMPORTANT**

These settings are currently defined directly in the `tc1d.py` script. To modify them, you need to edit the source code. Future versions may expose these parameters via command-line options.

In [ ]:
# run inversion
command = [
    "python", "tc1d/tc1d_cli.py",
    "--ero-type", "2",
    "--ero-option1", "0.0", "15.0",
    "--ero-option2", "6.0",
    "--ero-option3", "0.0", "15.0",
    "--ero-option4", "12.0",
    "--time", "12.0",
    "--dt", "20000",
    "--obs-age-file", "tc1d/sample_data.csv",
    "--inverse-mode", "MCMC"
]

subprocess.run(command)

In [ ]:
# Display plots
display(Image("mcmc_misfit.png"))
display(Image("mcmc_chains.png"))
display(Image("mcmc_corner.png"))
display(Image("mcmc_scatter_ero_option1_vs_ero_option3.png"))

### Interpretation of the MCMC Inversion Results

The following plots summarize the output of the Markov Chain Monte Carlo (MCMC) inversion performed with T<sub>c</sub>1D:

1. **Misfit Plot (`mcmc_misfit.png`)**  
   This scatter plot shows the misfit values associated with all sampled models.  
   - The y-axis is plotted on a log scale for clarity.  
   - The green dot marks the model with the lowest misfit.  
   This plot helps visualize the distribution and density of acceptable solutions.

2. **Chains Plot for Each Parameter (`mcmc_chains.png`)**  
   This figure displays the evolution of sampled values for each parameter over the course of the MCMC simulation.  
   - Each colored line corresponds to a walker.  
   - The x-axis represents the iteration step; the y-axis shows the parameter value.  
   - Well-mixed and overlapping chains suggest good convergence.  
   Visual inspection helps diagnose whether the chains are stuck or have reached stationarity.

3. **Corner Plot (`mcmc_corner.png`)**  
   This figure shows the marginal posterior distributions and pairwise joint distributions of the inverted parameters.  
   - The 1D histograms (top) represent the distribution of each parameter.  
   - The 2D contour plots reveal parameter correlations.  
   - The blue lines indicate the median values with 1σ uncertainties.  
   This plot provides a compact summary of uncertainty and correlation in parameter estimates.

4. **Scatter Plot of Sampled Parameter Space (`mcmc_scatter_ero_option1_vs_ero_option3.png`)**  
   This plot shows how the MCMC sampled the parameter space of two erosion parameters (`ero_option1` and `ero_option3`).  
   - The color scale represents misfit (darker = better fit).  
   - Histograms show the marginal distributions of each parameter.  
   - The red cross marks the best-fitting model.  
   This visualization is useful for identifying well-constrained regions and parameter trade-offs.

## Forward Modeling with Best-Fit Parameters

After completing the inversion, we can now run TC1D in forward mode using the best-fit parameters identified by MCMC.

This allows us to:

- Simulate the thermal evolution of the model based on these optimal parameters.
- Predict synthetic thermochronological ages.
- Compare these predicted ages to the original synthetic dataset.

This comparison provides a direct assessment of how well the inverted model reproduces the observed data and allows us to evaluate the quality of the inversion.

In [ ]:
# run forward
command = [
    "python", "tc1d/tc1d_cli.py",
    "--ero-type", "2",
    "--ero-option1", "2.49",# MCMC inversion should converge around this value
    "--ero-option2", "6.0",
    "--ero-option3", "6.12",# MCMC inversion should converge around this value
    "--ero-option4", "12.0",
    "--time", "12.0",
    "--dt", "20000",
    "--obs-age-file", "tc1d/sample_data.csv",
    "--save-plots",
]

subprocess.run(command)

display(Image("png/cooling_hist.png"))